In [1]:
import numpy as np;
import matplotlib.pyplot as plt;
import pandas as pd;
from matplotlib.pyplot import figure
import keras;
import tensorflow as tf;
from keras.preprocessing.sequence import TimeseriesGenerator;
from keras.models import Sequential
from keras.layers import LSTM, Dense
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler
import yfinance as yf
import datetime
from keras.models import clone_model

In [5]:
def fetch_stock_data(ticker_symbol, start_date, end_date):
    # Fetch data
    stock_data = yf.download(ticker_symbol, start=start_date, end=end_date)
    # Select only Close and Volume columns
    stock_data = stock_data[['Close']]

    # Separate the dataframes
    df_main = stock_data.iloc[:-4]        # All data minus the last 5 days
    df_future = stock_data.iloc[-5:]      # Only the last 5 days

    return df_main, df_future

def main():
    # List of companies to fetch data for
    companies = ["NVDA", "TSLA", "FDX", "ABNB", "AAPL", "GOOG", "ORCL", "IBM", "MSFT"]

    # Determine date range: approximately 6 months from today's date
    end_date = datetime.date.today()
    start_date = end_date - datetime.timedelta(days=180)

    all_data = {}           # Dictionary to store main DataFrames for each company
    all_future_data = {}    # Dictionary to store future DataFrames for each company

    for company in companies:
        print(f"Fetching data for {company} from {start_date} to {end_date}")
        df_main, df_future = fetch_stock_data(company, start_date, end_date)

        all_data[company] = df_main
        all_future_data[company + "_future"] = df_future

        print(df_main)
        print("-" * 50)

    return all_data, all_future_data

if __name__ == "__main__":
    all_data, all_future_data = main()
    locals().update(all_data)
    locals().update(all_future_data)


Fetching data for NVDA from 2023-04-30 to 2023-10-27
[*********************100%%**********************]  1 of 1 completed
                 Close
Date                  
2023-05-01  289.100006
2023-05-02  282.100006
2023-05-03  278.019989
2023-05-04  275.619995
2023-05-05  286.799988
...                ...
2023-10-16  460.950012
2023-10-17  439.380005
2023-10-18  421.959991
2023-10-19  421.010010
2023-10-20  413.869995

[121 rows x 1 columns]
--------------------------------------------------
Fetching data for TSLA from 2023-04-30 to 2023-10-27
[*********************100%%**********************]  1 of 1 completed
                 Close
Date                  
2023-05-01  161.830002
2023-05-02  160.309998
2023-05-03  160.610001
2023-05-04  161.199997
2023-05-05  170.059998
...                ...
2023-10-16  253.919998
2023-10-17  254.850006
2023-10-18  242.679993
2023-10-19  220.110001
2023-10-20  211.990005

[121 rows x 1 columns]
--------------------------------------------------
Fetching

In [6]:
# !pip install --quiet --upgrade tensorflow-federated

In [7]:
import tensorflow as tf
import tensorflow_federated as tff

# 1. Preprocessing Data for Federated Learning

def preprocess_for_federated_learning(company_data):
    # This function will convert your pandas dataframe to a format that TFF can use
    def dataframe_to_dataset(dataframe):
        dataframe = dataframe.copy()
        labels = dataframe.pop('Close')
        ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
        ds = ds.shuffle(buffer_size=len(dataframe))
        ds = ds.batch(5) # Batch size for your look_back
        return ds

    client_datasets = []
    for _, df in company_data.items():
        ds = dataframe_to_dataset(df)
        client_datasets.append(ds)
    return client_datasets

# 2. Model Definition

def create_lstm_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.LSTM(30, activation=tf.nn.leaky_relu, input_shape=(5,1)),
        tf.keras.layers.Dense(1)
    ])
    return model

# 3. Federated Learning with TFF

# Wrap the Keras model in TFF
def model_fn(client_datasets):
    keras_model = create_lstm_model()
    return tff.learning.models.from_keras_model(
        keras_model,
        input_spec=client_datasets[0].element_spec,
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.MeanSquaredError()]
    )

# Load and preprocess data
all_data, _ = main()
client_datasets = preprocess_for_federated_learning(all_data)

# Create iterative process and initialize the global model
iterative_process = tff.learning.algorithms.build_weighted_fed_avg(
    lambda: model_fn(client_datasets),
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0)
)
state = iterative_process.initialize()

# Federated Learning Iterations
NUM_ROUNDS = 10
for round_num in range(NUM_ROUNDS):
    state, metrics = iterative_process.next(state, client_datasets)
    print('round {:2d}, metrics={}'.format(round_num, metrics))



Fetching data for NVDA from 2023-04-30 to 2023-10-27
[*********************100%%**********************]  1 of 1 completed
                 Close
Date                  
2023-05-01  289.100006
2023-05-02  282.100006
2023-05-03  278.019989
2023-05-04  275.619995
2023-05-05  286.799988
...                ...
2023-10-16  460.950012
2023-10-17  439.380005
2023-10-18  421.959991
2023-10-19  421.010010
2023-10-20  413.869995

[121 rows x 1 columns]
--------------------------------------------------
Fetching data for TSLA from 2023-04-30 to 2023-10-27
[*********************100%%**********************]  1 of 1 completed
                 Close
Date                  
2023-05-01  161.830002
2023-05-02  160.309998
2023-05-03  160.610001
2023-05-04  161.199997
2023-05-05  170.059998
...                ...
2023-10-16  253.919998
2023-10-17  254.850006
2023-10-18  242.679993
2023-10-19  220.110001
2023-10-20  211.990005

[121 rows x 1 columns]
--------------------------------------------------
Fetching

ValueError: ignored

In [ ]:
#  !pip install --upgrade pip setuptools wheel

In [ ]:
!pip install tensorflow tensorflow_federated

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.8/104.8 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.6/721.6 kB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.7/73.7 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.7/33.7 MB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.8/257.8 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.4/405.4 kB 19.7 MB/s eta 0:00:00
     ━━━

In [ ]:
!pip install typing-extensions==4.5.0


ERROR: Could not find a version that satisfies the requirement tensorflow==4.8.0 (from versions: 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.11.1, 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0)
ERROR: No matching distribution found for tensorflow==4.8.0


In [ ]:
!pip install scipy==1.9.3

In [ ]:
!pip install scipy==1.9.3
import tensorflow as tf
import tensorflow_federated as tff


ImportError: ignored

In [ ]:
!pip uninstall fastapi


Found existing installation: fastapi 0.104.0
Uninstalling fastapi-0.104.0:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/fastapi-0.104.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/fastapi/*
Proceed (Y/n)? y
  Successfully uninstalled fastapi-0.104.0


In [ ]:
%%bash
pip install virtualenv
virtualenv venv
source venv/bin/activate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 17.2 MB/s eta 0:00:00
created virtual environment CPython3.10.12.final.0-64 in 1706ms
  creator CPython3Posix(dest=/content/venv, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==23.2.1, setuptools==68.2.0, wheel==0.41.2
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


In [ ]:
!pip install tensorflow tensorflow-federated


In [ ]:
# This is a makeshift solution to manually add the missing function.
import numpy as np
from scipy._lib._util import _lazywhere

def prod(a, axis=None, dtype=None, out=None, keepdims=np._NoValue, initial=np._NoValue, where=np._NoValue):
    return np.prod(a, axis=axis, dtype=dtype, out=out, keepdims=keepdims, initial=initial, where=where)

# Injecting the function to scipy's _util module
import scipy._lib._util
scipy._lib._util.prod = prod

# Now try importing tensorflow_federated
import tensorflow_federated as tff


NameError: ignored

In [ ]:
!pip uninstall tensorflow_federated -y


Found existing installation: tensorflow-federated 0.63.0
Uninstalling tensorflow-federated-0.63.0:
  Successfully uninstalled tensorflow-federated-0.63.0


In [ ]:
!pip install tensorflow==2.4.1
!pip install tensorflow_federated==0.18.0


ERROR: Could not find a version that satisfies the requirement tensorflow==2.4.1 (from versions: 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.11.1, 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0)
ERROR: No matching distribution found for tensorflow==2.4.1
  Using cached tensorflow_federated-0.18.0-py2.py3-none-any.whl (578 kB)
  Using cached absl_py-0.15.0-py3-none-any.whl (132 kB)
  Using cached attrs-19.3.0-py2.py3-none-any.whl (39 kB)
  Using cached cachetools-3.1.1-py2.py3-none-any.whl (11 kB)
  Using cached grpcio-1.32.0.tar.gz (20.8 MB)
  Preparing metadata (setup.py) ... done
  Using cached h5py-2.10.0.tar.gz (301 kB)
  Preparing metadata (setup.py) ... done
  Using cached jax-0.2.28.tar.gz (887 kB)
  Preparing metadata (setup.py) ... done
INF

In [ ]:
import tensorflow as tf
import tensorflow_federated as tff


ModuleNotFoundError: ignored

# Example of how to create federated learning:

In [ ]:
#@test {"skip": true}
!pip install --quiet --upgrade tensorflow-federated

In [ ]:
import tensorflow as tf
import tensorflow_federated as tff

In [ ]:
NUM_CLIENTS = 10
BATCH_SIZE = 20

def preprocess(dataset):

  def batch_format_fn(element):
    """Flatten a batch of EMNIST data and return a (features, label) tuple."""
    return (tf.reshape(element['pixels'], [-1, 784]),
            tf.reshape(element['label'], [-1, 1]))

  return dataset.batch(BATCH_SIZE).map(batch_format_fn)

In [ ]:
client_ids = sorted(emnist_train.client_ids)[:NUM_CLIENTS]
federated_train_data = [preprocess(emnist_train.create_tf_dataset_for_client(x))
  for x in client_ids
]

In [ ]:
def create_keras_model():
  initializer = tf.keras.initializers.GlorotNormal(seed=0)
  return tf.keras.models.Sequential([
      tf.keras.layers.Input(shape=(784,)),
      tf.keras.layers.Dense(10, kernel_initializer=initializer),
      tf.keras.layers.Softmax(),
  ])

In [ ]:
def model_fn():
  keras_model = create_keras_model()
  return tff.learning.models.from_keras_model(
      keras_model,
      input_spec=federated_train_data[0].element_spec,
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [ ]:
def initialize_fn():
  model = model_fn()
  return model.trainable_variables

In [ ]:
def next_fn(server_weights, federated_dataset):
  # Broadcast the server weights to the clients.
  server_weights_at_client = broadcast(server_weights)

  # Each client computes their updated weights.
  client_weights = client_update(federated_dataset, server_weights_at_client)

  # The server averages these updates.
  mean_client_weights = mean(client_weights)

  # The server updates its model.
  server_weights = server_update(mean_client_weights)

  return server_weights

In [ ]:
@tf.function
def client_update(model, dataset, server_weights, client_optimizer):
  """Performs training (using the server model weights) on the client's dataset."""
  # Initialize the client model with the current server weights.
  client_weights = model.trainable_variables
  # Assign the server weights to the client model.
  tf.nest.map_structure(lambda x, y: x.assign(y),
                        client_weights, server_weights)

  # Use the client_optimizer to update the local model.
  for batch in dataset:
    with tf.GradientTape() as tape:
      # Compute a forward pass on the batch of data
      outputs = model.forward_pass(batch)

    # Compute the corresponding gradient
    grads = tape.gradient(outputs.loss, client_weights)
    grads_and_vars = zip(grads, client_weights)

    # Apply the gradient using a client optimizer.
    client_optimizer.apply_gradients(grads_and_vars)

  return client_weights

In [ ]:
@tf.function
def server_update(model, mean_client_weights):
  """Updates the server model weights as the average of the client model weights."""
  model_weights = model.trainable_variables
  # Assign the mean client weights to the server model.
  tf.nest.map_structure(lambda x, y: x.assign(y),
                        model_weights, mean_client_weights)
  return model_weights

In [ ]:
federated_float_on_clients = tff.FederatedType(tf.float32, tff.CLIENTS)

In [ ]:
str(federated_float_on_clients)

In [ ]:
@tff.federated_computation(tff.FederatedType(tf.float32, tff.CLIENTS))
def get_average_temperature(client_temperatures):
  return tff.federated_mean(client_temperatures)

In [ ]:
str(get_average_temperature.type_signature)

In [ ]:
get_average_temperature([68.5, 70.3, 69.8])

In [ ]:
@tff.tf_computation(tf.float32)
def add_half(x):
  return tf.add(x, 0.5)

In [ ]:
str(add_half.type_signature)

In [ ]:
@tff.federated_computation(tff.FederatedType(tf.float32, tff.CLIENTS))
def add_half_on_clients(x):
  return tff.federated_map(add_half, x)

In [ ]:
str(add_half_on_clients.type_signature)

In [ ]:
@tff.tf_computation
def server_init():
  model = model_fn()
  return model.trainable_variables

In [ ]:
@tff.federated_computation
def initialize_fn():
  return tff.federated_value(server_init(), tff.SERVER)

In [ ]:
whimsy_model = model_fn()
tf_dataset_type = tff.SequenceType(whimsy_model.input_spec)

In [ ]:
str(tf_dataset_type)

In [ ]:
model_weights_type = server_init.type_signature.result

In [ ]:
str(model_weights_type)

In [ ]:
@tff.tf_computation(tf_dataset_type, model_weights_type)
def client_update_fn(tf_dataset, server_weights):
  model = model_fn()
  client_optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
  return client_update(model, tf_dataset, server_weights, client_optimizer)

In [ ]:
@tff.tf_computation(model_weights_type)
def server_update_fn(mean_client_weights):
  model = model_fn()
  return server_update(model, mean_client_weights)

In [ ]:
federated_server_type = tff.FederatedType(model_weights_type, tff.SERVER)
federated_dataset_type = tff.FederatedType(tf_dataset_type, tff.CLIENTS)

In [ ]:
@tff.federated_computation(federated_server_type, federated_dataset_type)
def next_fn(server_weights, federated_dataset):
  # Broadcast the server weights to the clients.
  server_weights_at_client = tff.federated_broadcast(server_weights)

  # Each client computes their updated weights.
  client_weights = tff.federated_map(
      client_update_fn, (federated_dataset, server_weights_at_client))

  # The server averages these updates.
  mean_client_weights = tff.federated_mean(client_weights)

  # The server updates its model.
  server_weights = tff.federated_map(server_update_fn, mean_client_weights)

  return server_weights

In [ ]:
federated_algorithm = tff.templates.IterativeProcess(
    initialize_fn=initialize_fn,
    next_fn=next_fn
)

In [ ]:
str(federated_algorithm.initialize.type_signature)

In [ ]:
str(federated_algorithm.next.type_signature)

In [ ]:
central_emnist_test = emnist_test.create_tf_dataset_from_all_clients()
central_emnist_test = preprocess(central_emnist_test)

In [ ]:
def evaluate(server_state):
  keras_model = create_keras_model()
  keras_model.compile(
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
  )
  keras_model.set_weights(server_state)
  keras_model.evaluate(central_emnist_test)

In [ ]:
server_state = federated_algorithm.initialize()
evaluate(server_state)

In [ ]:
for round in range(15):
  server_state = federated_algorithm.next(server_state, federated_train_data)

In [ ]:
evaluate(server_state)

## Testing Code:

In [ ]:
import yfinance as yf
import datetime
import tensorflow as tf
import tensorflow_federated as tff

def fetch_stock_data(ticker_symbol, start_date, end_date):
    stock_data = yf.download(ticker_symbol, start=start_date, end=end_date)
    stock_data = stock_data[['Close', 'Volume']]
    df_main = stock_data.iloc[:-4]
    df_future = stock_data.iloc[-5:]
    return df_main, df_future

def main():
    companies = ["NVDA", "TSLA", "FDX", "ABNB", "AAPL", "GOOG", "ORCL", "IBM", "MSFT"]
    end_date = datetime.date.today()
    start_date = end_date - datetime.timedelta(days=180)
    all_data = {}
    all_future_data = {}
    for company in companies:
        df_main, df_future = fetch_stock_data(company, start_date, end_date)
        all_data[company] = df_main
        all_future_data[company + "_future"] = df_future
    return all_data, all_future_data

def preprocess_for_federated_learning(company_data):
    def dataframe_to_dataset(dataframe):
        dataframe = dataframe.copy()
        labels = dataframe.pop('Close')
        ds = tf.data.Dataset.from_tensor_slices((dataframe.values, labels))
        ds = ds.shuffle(buffer_size=len(dataframe))
        ds = ds.batch(10)  # Adjusted batch size for experimentation
        return ds

    client_datasets = []
    for _, df in company_data.items():
        ds = dataframe_to_dataset(df)
        client_datasets.append(ds)
    return client_datasets

def create_lstm_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.LSTM(50, activation=tf.nn.leaky_relu, input_shape=(10, 1)),
        tf.keras.layers.Dense(1)
    ])
    return model

def dataframe_to_dataset(dataframe):
    dataframe = dataframe.copy()
    labels = dataframe.pop('Close')
    ds = tf.data.Dataset.from_tensor_slices((dataframe.values.reshape(-1, 10, 1), labels))
    ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(10)  # Adjusted batch size for experimentation
    return ds

def model_fn():
    keras_model = create_lstm_model()
    # Note: We can't access a specific client dataset here, so we'll use a
    # generic input spec. This is an example, adjust based on your data structure.
    input_spec = (tf.TensorSpec(shape=(None, 10, 1), dtype=tf.float32),
                  tf.TensorSpec(shape=(None,), dtype=tf.float32))
    return tff.learning.from_keras_model(
        keras_model,
        input_spec=input_spec,
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.MeanSquaredError()]
    )

all_data, _ = main()
client_datasets = preprocess_for_federated_learning(all_data)

# Initialize the learning rate schedule and optimizer without the graph context.
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=10000,
    decay_rate=0.9)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

iterative_process = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.Adam(learning_rate=lr_schedule),
    server_optimizer_fn=lambda: tf.keras.optimizers.Adam(learning_rate=lr_schedule)
)
state = iterative_process.initialize()

NUM_ROUNDS = 10
for round_num in range(NUM_ROUNDS):
    state, metrics = iterative_process.next(state, client_datasets)
    print('round {:2d}, metrics={}'.format(round_num, metrics))

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


AttributeError: ignored

ImportError: ignored

In [ ]:
!pip install --quiet --upgrade tensorflow-federated

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastapi 0.104.0 requires typing-extensions>=4.8.0, but you have typing-extensions 4.5.0 which is incompatible.
google-colab 1.0.0 requires portpicker==1.5.2, but you have portpicker 1.6.0 which is incompatible.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.20 which is incompatible.


In [ ]:
!pip install portpicker==1.5.2


  Attempting uninstall: portpicker
    Found existing installation: portpicker 1.6.0
    Uninstalling portpicker-1.6.0:
      Successfully uninstalled portpicker-1.6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-federated 0.63.0 requires portpicker~=1.6, but you have portpicker 1.5.2 which is incompatible.


In [ ]:
!pip install fastapi kaleido python-multipart uvicorn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.13.0 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.8.0 which is incompatible.
tensorflow-federated 0.63.0 r

In [ ]:
!pip install sqlalchemy --upgrade


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 8.3 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.4.20
    Uninstalling SQLAlchemy-1.4.20:
      Successfully uninstalled SQLAlchemy-1.4.20
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-vizier 0.1.11 requires sqlalchemy<=1.4.20,>=1.4, but you have sqlalchemy 2.0.22 which is incompatible.
tensorflow-federated 0.63.0 requires portpicker~=1.6, but you have portpicker 1.5.2 which is incompatible.
tensorflow-federated 0.63.0 requires typing-extensions==4.5.*,>=4.5.0, but you have typing-extensions 4.8.0 which is incompatible.
